In [12]:
import arcpy
from arcpy import env
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import pysal as ps
import glob
import shutil
import pathlib
from IPython.display import display

In [13]:
tempDir = "C:\TempAnalysis"
dataDir = "C:\TestData"

In [14]:
def buildTempWorkspace(tempDir):
    if not os.path.exists(tempDir):
        os.mkdir(tempDir)
        print("Directory " ,tempDir,  " Created.")
    else:    
        print("Directory " ,tempDir,  " already exists.")

In [15]:
def buildAnalysisDatabaseList(dataDir):
    env.workspace = dataDir
    fgdbList = arcpy.ListWorkspaces("*", "FileGDB")
    return fgdbList

In [16]:
def buildAnalysisTables(fgdb,tempDir):
    env.workspace = fgdb        
    fcpList = []
    dsList = arcpy.ListDatasets(feature_type='feature')
    dsList = [''] + dsList if dsList is not None else []
    for ds in dsList:
        for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
            path = os.path.join(fgdb, ds, fc)
            fcpList.append(path)
    fgdbDirName = os.path.splitext(os.path.basename(fgdb))[0]
    fgdbDirPath = os.path.join(tempDir,fgdbDirName)
    if not os.path.exists(fgdbDirPath):
        os.mkdir(fgdbDirPath)
        print("Directory " ,fgdbDirPath,  " created. ")
    else:    
        print("Directory " ,fgdbDirPath,  " already exists.")
    for fcp in fcpList:
        print(fcp)
        arcpy.TableToTable_conversion(fcp, fgdbDirPath,os.path.splitext(os.path.basename(fcp))[0])

In [17]:
def buildDBFPList(fgdb):
    dbfpList = []
    fgdbDirName = os.path.splitext(os.path.basename(fgdb))[0]
    fgdbDirPath = os.path.join(tempDir,fgdbDirName)
    for p in pathlib.Path(fgdbDirPath).glob('*.dbf'):
        if p.is_file():
            dbfpList.append(str(p))
    return(dbfpList)

In [18]:
def dbf2DF(dbfp, upper=True):
    db = ps.open(dbfp)
    d = {col: db.by_col(col) for col in db.header}
    pandasDF = pd.DataFrame(d)
    if upper == True:
        pandasDF.columns = map(str.upper, db.header) 
    db.close()
    return pandasDF

In [19]:
def dbfIterate(dbfpList):
    for dbfp in dbfpList:
        print(os.path.splitext(os.path.basename(dbfp))[0])
        thisDF = (dbf2DF(dbfp, upper=True))
        cleanThisDF = thisDF.dropna()
        display(cleanThisDF)

In [20]:
def fgdbIterate(fgdbList):
    print("Found the following FGDBs: ",fgdbList)
    for fgdb in fgdbList:
        buildAnalysisTables(fgdb,tempDir)
        dbfIterate(buildDBFPList(fgdb))

In [21]:
def cleanupTempWorkspace(tempDir):
    if not os.path.exists(tempDir):
        print("Directory " ,tempDir,  " not present.")
    else:
        shutil.rmtree(tempDir)
        print("Directory " ,tempDir,  " cleaned up.")

In [22]:
buildTempWorkspace(tempDir)
buildAnalysisDatabaseList(dataDir)
fgdbIterate(buildAnalysisDatabaseList(dataDir))
cleanupTempWorkspace(tempDir)

Directory  C:\TempAnalysis  Created.
Found the following FGDBs:  ['C:\\TestData\\CO_Wetlands.gdb']
Directory  C:\TempAnalysis\CO_Wetlands  created. 
C:\TestData\CO_Wetlands.gdb\Colorado
C:\TestData\CO_Wetlands.gdb\Wetlands\CO_Wetlands
C:\TestData\CO_Wetlands.gdb\Wetlands\CO_Wetlands_Project_Metadata
C:\TestData\CO_Wetlands.gdb\Wetlands\CO_Wetlands_Historic_Map_Info
C:\TestData\CO_Wetlands.gdb\Riparian\CO_Riparian
C:\TestData\CO_Wetlands.gdb\Riparian\CO_Riparian_Project_Metadata
Colorado


,ST_NAME1,ST_ABBV,AREAACRES,SHAPE_LENG,SHAPE_AREA
0,Colorado,CO,7.194448e+07,2.175185e+06,2.911490e+11


CO_Riparian


,ATTRIBUTE,WETLAND_TY,ACRES,SHAPE_LENG,SHAPE_AREA
0,Rp1FO6MD,Forested/Shrub Riparian,1.462499,484.645804,5918.522850
1,Rp1FO6MD,Forested/Shrub Riparian,0.336117,160.390327,1360.216950
2,Rp1SS6SC,Forested/Shrub Riparian,1.131145,606.281514,4577.582710
3,Rp1SS6SC,Forested/Shrub Riparian,0.391082,215.278821,1582.653000
4,Rp1SS6SC,Forested/Shrub Riparian,1.191667,330.338272,4822.504660
5,Rp1SS6SC,Forested/Shrub Riparian,2.323999,526.828122,9404.889100
6,Rp1SS6SC,Forested/Shrub Riparian,1.299086,392.565615,5257.215201
7,Rp1SS6SC,Forested/Shrub Riparian,1.251092,464.405656,5062.987737
8,Rp1SS6SC,Forested/Shrub Riparian,1.554921,560.881531,6292.543900
9,Rp1SS6SC,Forested/Shrub Riparian,0.350481,212.672290,1418.345200


CO_Riparian_Project_Metadata


...

CO_Wetlands_Historic_Map_Info


,PDF_NAME,PDF_HYPERL,SHAPE_LENG,SHAPE_AREA
0,albuquerque-b.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/alb...,204485.468692,1.235733e+09
1,durango-b.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/dur...,597757.500427,8.560503e+09
2,aztec-c.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/azt...,204485.102369,1.235734e+09
3,price-a.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/pri...,469312.099975,2.370353e+09
4,sterling-a.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/ste...,280691.138193,4.723761e+09
5,cheyenne-a.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/che...,361482.932970,2.333893e+09
6,OKMapNarrativeDelhartNE.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/OKM...,204486.027551,1.235731e+09
7,denver-a.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/den...,281897.253598,4.757960e+09
8,denver-b.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/den...,281898.857309,4.758002e+09
9,denver-c.pdf,http://www.fws.gov/wetlands/Data/HisMapRep/den...,283093.628390,4.791731e+09


CO_Wetlands_Project_Metadata


,PROJECT_NA,STATUS,SUPPMAPINF,DATA_SOURC,IMAGE_YR,IMAGE_DATE,IMAGE_SCAL,ALL_SCALES,EMULSION,COMMENTS,SOURCE_TYP,DATA_CAT,SHAPE_LENG,SHAPE_AREA
0,Dolores Point North,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R06...,None,1986,xx/80 to xx/86,58000,"1:58,000",CIR,Photo dates ranged from 1980 to 1986; NWI Ver...,CIR,Updated,49562.896961,1.509076e+08
1,Burnt Cabin Creek,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R06...,None,1986,xx/80 to xx/86,58000,"1:58,000",CIR,Photo dates ranged from 1980 to 1986; NWI Ver...,CIR,Updated,49855.919896,1.529626e+08
2,Crouse Reservoir,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R06...,None,1983,08/83,58000,1:58K,CIR,NWI Version 2 derived by supplementing single-...,CIR,Legacy,3714.149324,3.124783e+01
3,Dragon,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R06...,None,1985,xx/85,58000,1:58K,CIR,NWI Version 2 derived by supplementing single-...,CIR,Legacy,49225.062229,1.485520e+08
4,Papoose Canyon,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R06...,None,1986,xx/80 to xx/86,58000,"1:58,000",CIR,Photo dates ranged from 1980 to 1986; NWI Ver...,CIR,Updated,49891.377581,1.532130e+08
5,San Juan,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R02...,NWI-REG2,2001,"09/00, 05/01",15840,1:15840,CIR,NWI Version 2 derived by supplementing single-...,CIR,Modern,188378.880791,9.268585e+08
6,Cheyenne South,Digital,None,NWIC,1980,08/80,58000,1:58K,CIR,COMP; NWI Version 2 derived by supplementing ...,CIR,Legacy,48833.403256,1.458490e+08
7,Arcola,Digital,None,NWIC,1980,09/80,58000,1:58K,CIR,COMP; NWI Version 2 derived by supplementing ...,CIR,Legacy,48834.580599,1.458572e+08
8,Altvan,Digital,None,NWIC,1980,09/80,58000,1:58K,CIR,COMP; NWI Version 2 derived by supplementing ...,CIR,Legacy,48834.985141,1.458598e+08
9,Pinon Canyon,Digital,http://www.fws.gov/wetlands/Data/SupMapInf/R06...,NWIC-REG6,1998,xx/98,40000,1:40K,BW,None; NWI Version 2 derived by supplementing ...,BW,Updated,243413.219456,2.606857e+09


Directory  C:\TempAnalysis  cleaned up.
